In [1]:
import pandas as pd
import sys
import os
import numpy as np

In [2]:
df = pd.read_csv('../datasets/dataset_processed/data_orders_offers.csv')

In [3]:
df.rename(columns={ 'order_status_key': 'order_status'}
            , inplace=True)

In [4]:
df['order_status_txt'] = df['order_status'].map({
    4: 'cancelled_client',
    9: 'cancelled_system'})

In [5]:
df['is_driver_assigned_key_txt'] = df['is_driver_assigned_key'].map({
    0: 'not_assigned',
    1: 'assigned'})

In [6]:
df

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,order_status,is_driver_assigned_key,cancellations_time_in_seconds,offer_id,order_status_txt,is_driver_assigned_key_txt
0,18:08:07,-0.978916,51.456173,60.0,3000583041974,4,1,198.0,3.000510e+11,cancelled_client,assigned
1,20:57:32,-0.950385,51.456843,NaN,3000583116437,4,0,128.0,3.000510e+11,cancelled_client,not_assigned
2,20:57:32,-0.950385,51.456843,NaN,3000583116437,4,0,128.0,3.000510e+11,cancelled_client,not_assigned
3,20:57:32,-0.950385,51.456843,NaN,3000583116437,4,0,128.0,3.000510e+11,cancelled_client,not_assigned
4,12:07:50,-0.969520,51.455544,477.0,3000582891479,4,1,46.0,3.000510e+11,cancelled_client,assigned
...,...,...,...,...,...,...,...,...,...,...,...
34369,13:11:35,-0.975372,51.457846,NaN,3000599186906,4,0,36.0,3.000516e+11,cancelled_client,not_assigned
34370,13:13:55,-0.975372,51.457846,NaN,3000599186962,4,0,4.0,NaN,cancelled_client,not_assigned
34371,13:17:21,-0.972926,51.457693,60.0,3000599187034,4,1,34.0,3.000516e+11,cancelled_client,assigned
34372,13:16:28,-0.975372,51.457846,NaN,3000599187024,4,0,6.0,NaN,cancelled_client,not_assigned


# Build up distribution of orders according to reasons for failure: cancellations before and after driver assignment, and reasons for order rejection. Analyse the resulting plot. Which category has the highest number of orders?

In [7]:
df_grouped = df.groupby(['order_status_txt', 'is_driver_assigned_key_txt']).agg(
    count=('order_gk', 'count'),
).reset_index()

In [8]:
import plotly.express as px

In [9]:
fig = px.bar(
    df_grouped,
    x='order_status_txt',         
    y='count',                     
    color='is_driver_assigned_key_txt',  
    barmode='group',               
    title='status order and driver assignment',
    labels={'count': 'N. orders', 'order_status_txt': 'Cancelled status'},
    color_discrete_map={          
        'assigned': '#1f77b4',
        'not_assigned': '#ff7f0e'
    }
)
fig.update_layout(
    legend_title_text='Assignment Status',
    xaxis_title='Cancelled Status',
    yaxis_title='Total Orders',
    hovermode='x unified'
)

fig.show()

## In this case we can look that the highest number of orders are the one who the driver was not assigned and cancelled by the client

***

# Plot the distribution of failed orders by hours. Is there a trend that certain hours have an abnormally high proportion of one category or another? What hours are the biggest fails? How can this be explained?

In [10]:
df['order_datetime'] = pd.to_datetime(df['order_datetime'])


/tmp/ipykernel_7206/127938464.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34374 entries, 0 to 34373
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_datetime                 34374 non-null  datetime64[ns]
 1   origin_longitude               34374 non-null  float64       
 2   origin_latitude                34374 non-null  float64       
 3   m_order_eta                    8838 non-null   float64       
 4   order_gk                       34374 non-null  int64         
 5   order_status                   34374 non-null  int64         
 6   is_driver_assigned_key         34374 non-null  int64         
 7   cancellations_time_in_seconds  23397 non-null  float64       
 8   offer_id                       31268 non-null  float64       
 9   order_status_txt               34374 non-null  object        
 10  is_driver_assigned_key_txt     34374 non-null  object        
dtypes: datetime64[n

In [12]:
fig = px.histogram(
    df,
    x='order_datetime',
    title='Orders by time of day',
    labels={'count': 'N. orders', 'order_datetime': 'Time of Day'},
    color='order_status_txt',
    color_discrete_map={
        'cancelled_client': '#1f77b4',
        'cancelled_system': '#ff7f0e'
    },
    histnorm='probability density',
    category_orders={'order_datetime': [f'{i:02d}:00' for i in range(24)]}
)
fig.show()

# Plot the average time to cancellation with and without driver, by the hour. If there are any outliers in the data, it would be better to remove them. Can we draw any conclusions from this plot?

In [22]:
df[df['cancellations_time_in_seconds'].isna()]

,order_datetime,origin_longitude,origin_latitude,m_order_eta,order_gk,order_status,is_driver_assigned_key,cancellations_time_in_seconds,offer_id,order_status_txt,is_driver_assigned_key_txt
13,2025-06-11 21:24:45,-0.967605,51.458236,NaN,3000583140877,9,0,NaN,NaN,cancelled_system,not_assigned
14,2025-06-11 21:21:23,-0.947011,51.456380,NaN,3000583117054,9,0,NaN,3.000510e+11,cancelled_system,not_assigned
15,2025-06-11 21:21:23,-0.947011,51.456380,NaN,3000583117054,9,0,NaN,3.000510e+11,cancelled_system,not_assigned
16,2025-06-11 21:21:23,-0.947011,51.456380,NaN,3000583117054,9,0,NaN,3.000510e+11,cancelled_system,not_assigned
17,2025-06-11 21:21:23,-0.947011,51.456380,NaN,3000583117054,9,0,NaN,3.000510e+11,cancelled_system,not_assigned
...,...,...,...,...,...,...,...,...,...,...,...
34293,2025-06-11 08:04:58,-0.976793,51.462002,NaN,3000554721763,9,0,NaN,NaN,cancelled_system,not_assigned
34295,2025-06-11 08:08:36,-0.972801,51.478548,NaN,3000554721897,9,0,NaN,3.000507e+11,cancelled_system,not_assigned
34296,2025-06-11 08:08:36,-0.972801,51.478548,NaN,3000554721897,9,0,NaN,3.000507e+11,cancelled_system,not_assigned
34297,2025-06-11 23:33:46,-0.964696,51.445968,NaN,3000555121226,9,0,NaN,NaN,cancelled_system,not_assigned


In [ ]:
df['cancellations_time_in_seconds'] = df['cancellations_time_in_seconds'].fillna(0)

In [36]:
df.groupby('order_status_txt').agg(
    avg_cancellations_time=('cancellations_time_in_seconds', 'mean'),
    count=('order_gk', 'count')
).reset_index().sort_values(by='avg_cancellations_time', ascending=False)

,order_status_txt,avg_cancellations_time,count
0,cancelled_client,165.655084,23397
1,cancelled_system,0.000000,10977
